# Image Processing SS 16 - Assignment - 09

### Deadline is 29.6.2016 at 16:00 o'clock

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io.wavfile
from skimage.data import astronaut
from skimage.color import rgb2gray

from __future__ import division
import random
try:
    from StringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
    
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
    
    
from numpy.fft import fft2 as numpy_fft2, ifft2 as numpy_ifft2
from scipy.fftpack import dct
from PIL import Image
import itertools
import IPython
import zipfile
pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

# Exercise 1 - Hadamard Matrix - 5 Points



In [ ]:
def hadamard_matrix(n):
    """Returns the Hadamard matrix. N is a power of two."""
    # your code here
    return np.zeros((n, n))

plt.subplot(121)
plt.imshow(hadamard_matrix(8), cmap='gray')
plt.subplot(122)
plt.imshow(hadamard_matrix(32), cmap='gray')
plt.show()

In [ ]:
def get_chess_board(n=8, field_size=8):
    board = np.zeros((n*field_size, n*field_size))
    s = field_size
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i*s:(i+1)*s, j*s:(j+1)*s] = 1
    return board

def get_sinus_board(n=8, field_size=8, shift=0):
    img_size = n*field_size
    linsp = np.linspace(-shift, np.pi*n - shift, img_size).reshape((img_size, 1))
    return np.dot(np.sin(linsp), np.sin(linsp.T))



In [ ]:
chess_board = get_chess_board()
chess_board_roll = np.roll(np.roll(chess_board, 4, axis=0), 4, axis=1)
sinus_board = get_sinus_board()
sinus_board_roll = get_sinus_board(shift=4)
plt.subplot(221)
plt.imshow(chess_board, cmap='gray', interpolation='nearest')
plt.subplot(222)
plt.imshow(sinus_board, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll, cmap='gray', interpolation='nearest')
plt.subplot(224)
plt.imshow(sinus_board_roll, cmap='gray')
plt.show()

In [ ]:
# Plot the 2d hadamard transformation of the chess boards and sinus boards.
# The 2D Hadamard Transformation can be computed similiar to the 2D DFT:
# H * I * H, where I is the image, H is the hadamard-matrix and * is the matrix multiplication.

H = hadamard_matrix(64)

chess_board_H = chess_board                         # your code here
chess_board_roll_H = chess_board_roll
sinus_board_H = sinus_board
sinus_board_roll_H = sinus_board_roll

plt.subplot(221)
plt.imshow(chess_board_H, cmap='gray', interpolation='nearest')
plt.subplot(222)
plt.imshow(sinus_board_H, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll_H, cmap='gray', interpolation='nearest')
plt.subplot(224)
plt.imshow(sinus_board_roll_H, cmap='gray')
plt.show()

In [ ]:
plt.imshow(np.dot(np.dot(H, chess_board_roll), H))

# Plot the fourier transformation of chess boards and sinus boards. 
# You can use some functions from np.ftt
chess_board_ft = chess_board                       # your code here
chess_board_roll_ft = chess_board_roll 
sinus_board_ft = sinus_board
sinus_board_roll_ft = sinus_board_roll

plt.subplot(221)
plt.imshow(chess_board_ft, cmap='gray', interpolation='nearest')
plt.subplot(222)
plt.imshow(sinus_board_ft, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll_ft, cmap='gray', interpolation='nearest')
plt.subplot(224)
plt.imshow(sinus_board_roll_ft, cmap='gray')
plt.show()

# Exercise 2 - Implement Hadamard Filter Bank - 5 Points

In [ ]:
def get_black_holes_colliding_data():
    """
    The LIGO experiment is measuring the gravitational waves.
    We will analyse the frequiencies of two colliding black holes.
    
    Here is an explanation to gravitational waves (if you interessted):
        https://www.ligo.caltech.edu/page/what-are-gw
    This is the original paper:
        https://physics.aps.org/featured-article-pdf/10.1103/PhysRevLett.116.061102
    We will use the data from the Hanford detector. 
    """
    url = "https://losc.ligo.org/s/events/GW150914/P150914/fig1-observed-H.txt"
    f = urllib2.urlopen(url)
    data = []
    for i, byte_line in enumerate(f.readlines()):
        if i == 0:
            continue
        line = byte_line.decode('utf-8').rstrip("\n")
        data.append(float(line.split(" ")[1]))
    return np.array(data)

In [ ]:
data = get_black_holes_colliding_data()

In [ ]:
plt.figure(figsize=(12, 3))
plt.ylabel("intensity")
plt.xlabel("time")
plt.plot(data)
plt.show()

In [ ]:
# listen to the two colliding black holes
IPython.display.Audio(data=data, rate=1000)

In [ ]:
# pad it to the power of two
data_padded = np.zeros((4096)) 
data_padded[:len(data)] = data
nb_padded = len(data_padded) - len(data)

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d, convolve1d
def hadamard_filter_bank(I):
    """Runs the hadamard filter bank as described in the lecture.
       Returns a list of list of coefficents. The first list are the coefficents from 
       the first bank and the last list the coefficents from the last bank (the dc part).
       `I` is allways a power of two.       
    """
    # your code here
    return [[1, 2, 0, 1]*100, [1, 0]*100, [1]*100]
    

def visualise_filter_bank(coefficents):
    """
    Builds one matrix with the lower coefficents repeated.
    """
    vis_mat = np.zeros((len(coefficents), len(coefficents[0])))
    for i, coeffs in enumerate(coefficents):
        arr = np.array(coeffs)
        vis_mat[i] = np.repeat(arr, 2**i, axis=-1)[:len(vis_mat[i])]
    return vis_mat

In [ ]:
data_h = hadamard_filter_bank(data_padded)

In [ ]:
vis = visualise_filter_bank(data_h)
plt.figure(figsize=(12, 4))
plt.plot(data[:3000])
plt.ylabel("intensity")
plt.yticks([])
plt.xlabel("time")
plt.show()

plt.figure(figsize=(15, 4))
plt.imshow(vis.repeat(40, axis=0)[:, :-int(nb_padded / 2)], cmap=plt.cm.viridis)
plt.colorbar()
plt.ylabel("frequiencies")
plt.yticks([])
plt.xlabel("time")
plt.show()